In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()


True

In [8]:

ds = tfds.load(name="imagenette", with_info=True, split=["train","validation"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(32)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


In [10]:
class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.layer1 = layers.Dense(512, activation='relu')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.layer2 = layers.Dense(128, activation='relu')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.layer3 = layers.Dense(10, activation='softmax')
    def call(self,inputs, training=True):
        
        x = layers.Flatten()(inputs)
        #print(x)
        x = self.layer1(x)
        x = self.bn1(x, training = training)
        x = self.layer2(x)
        x = self.bn2(x, training = training)
        return self.layer3(x)
        self.conv1 = CustomConvLayer(masks['mask_conv_1'], masks['mask_conv_1'], biases['bias_conv_1'], 1)
        self.pool1 = CustomPoolLayer()
        self.conv2 = CustomConvLayer(masks['mask_conv_2'], masks['mask_conv_2'], biases['bias_conv_2'], 1)
        self.pool2 = CustomPoolLayer()
        self.conv3 = CustomConvLayer(masks['mask_conv_3'], masks['mask_conv_3'], biases['bias_conv_3'], 1)
        self.pool3 = CustomPoolLayer()
        self.conv4 = CustomConvLayer(masks['mask_conv_4'], masks['mask_conv_4'], biases['bias_conv_4'], 1)
        self.pool4 = CustomPoolLayer()
        self.dense1 = layers.Dense(1024, activation='relu')
        self.dense2 = layers.Dense(512, activation='relu')
        self.dense3 = layers.Dense(10, activation='softmax')
    def call(self,inputs):
        x = self.conv1(inputs)
        print(x)
        x = self.pool1(x)
        print(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.conv4(x)
        x = self.pool4(x)
        print(x)
        x = layers.Flatten()(x)
        #print(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3(x)

model = CustomModel()

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.00001),
    metrics=['accuracy'],
)

model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=15,
    #validation_data=ds_test,
)

AttributeError: in converted code:

    <ipython-input-3-492fa2eefa6e>:30 call  *
        x = self.conv1(inputs)

    AttributeError: 'CustomModel' object has no attribute 'conv1'


In [4]:
for example in ds_train.take(1):  # Only take a single example
    print(example['image'].shape)
    image, label = example["image"], example["label"]
    plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
    print("Label: %d" % label.numpy())

TypeError: tuple indices must be integers or slices, not str

In [5]:
ds_train = ds_train.repeat().shuffle(1024).batch(32)

# prefetch will enable the input pipeline to asynchronously fetch batches while
# your model is training.
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
tfds.show_examples(ds_train[1] ,ds_train[0]['train'])

TypeError: 'DatasetV1Adapter' object is not subscriptable

In [15]:
model.summary()

Model: "custom_model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
custom_conv_layer_20 (Custom multiple                  448       
_________________________________________________________________
custom_pool_layer_20 (Custom multiple                  0         
_________________________________________________________________
custom_conv_layer_21 (Custom multiple                  4640      
_________________________________________________________________
custom_pool_layer_21 (Custom multiple                  0         
_________________________________________________________________
custom_conv_layer_22 (Custom multiple                  18496     
_________________________________________________________________
custom_pool_layer_22 (Custom multiple                  0         
_________________________________________________________________
custom_conv_layer_23 (Custom multiple               

In [17]:
model.predict_on_batch(ds_train)

Tensor("custom_model_5/custom_conv_layer_20/Relu:0", shape=(32, 224, 224, 16), dtype=float32)
Tensor("custom_model_5/custom_pool_layer_20/MaxPool2d:0", shape=(32, 112, 112, 16), dtype=float32)
Tensor("custom_model_5/custom_pool_layer_23/MaxPool2d:0", shape=(32, 14, 14, 128), dtype=float32)


<tf.Tensor: shape=(32, 10), dtype=float32, numpy=
array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0.,

In [6]:
weights = {
    'weights_conv_1': tf.Variable(tf.random.normal([3, 3, 3, 16])),
    'weights_conv_2': tf.Variable(tf.random.normal([3, 3, 16, 32])),
    'weights_conv_3': tf.Variable(tf.random.normal([3, 3, 32, 64])),
    'weights_conv_4': tf.Variable(tf.random.normal([3, 3, 64, 128])),
    'weights_conv_5': tf.Variable(tf.random.normal([3, 3, 12, 24])),
    'weights_conv_6': tf.Variable(tf.random.normal([3, 3, 24, 24])),
    'weights_conv_7': tf.Variable(tf.random.normal([3, 3, 24, 24])),
    'weights_conv_8': tf.Variable(tf.random.normal([3, 3, 24, 48])),
    'weights_conv_9': tf.Variable(tf.random.normal([3, 3, 48, 48])),
    'weights_conv_10': tf.Variable(tf.random.normal([3, 3, 48, 48])),
    'weights_conv_11': tf.Variable(tf.random.normal([3, 3, 48, 48])),
    'weights_conv_12': tf.Variable(tf.random.normal([3, 3, 48, 48])),
    'weights_conv_13': tf.Variable(tf.random.normal([3, 3, 48, 48])),


}

masks = {
    
    'mask_conv_1': tf.Variable(tf.ones([3, 3, 3, 16]), trainable=False),
    'mask_conv_2': tf.Variable(tf.ones([3, 3, 16, 32]), trainable=False),
    'mask_conv_3': tf.Variable(tf.ones([3, 3, 32, 64]), trainable=False),
    'mask_conv_4': tf.Variable(tf.ones([3, 3, 64, 128]), trainable=False),
    'mask_conv_5': tf.Variable(tf.ones([3, 3, 12, 24]), trainable=False),
    'mask_conv_6': tf.Variable(tf.ones([3, 3, 24, 24]), trainable=False),
    'mask_conv_7': tf.Variable(tf.ones([3, 3, 24, 24]), trainable=False),
    'mask_conv_8': tf.Variable(tf.ones([3, 3, 24, 48]), trainable=False),
    'mask_conv_9': tf.Variable(tf.ones([3, 3, 48, 48]), trainable=False),
    'mask_conv_10': tf.Variable(tf.ones([3, 3, 48, 48]), trainable=False),
    'mask_conv_11': tf.Variable(tf.ones([3, 3, 48, 48]), trainable=False),
    'mask_conv_12': tf.Variable(tf.ones([3, 3, 48, 48]), trainable=False),
    'mask_conv_13': tf.Variable(tf.ones([3, 3, 48, 48]), trainable=False),
    # 224x224 input --> 5 maxpool layers --> 

}

biases = {
    #output depth
    'bias_conv_1': tf.Variable(tf.zeros([16]), trainable=False),
    'bias_conv_2': tf.Variable(tf.zeros([32]), trainable=False),
    'bias_conv_3': tf.Variable(tf.zeros([64]), trainable=False),
    'bias_conv_4': tf.Variable(tf.zeros([128]), trainable=False),
    'bias_conv_5': tf.Variable(tf.zeros([24]), trainable=False),
    'bias_conv_6': tf.Variable(tf.zeros([24]), trainable=False),
    'bias_conv_7': tf.Variable(tf.zeros([24]), trainable=False),
    'bias_conv_8': tf.Variable(tf.zeros([48]), trainable=False),
    'bias_conv_9': tf.Variable(tf.zeros([48]), trainable=False),
    'bias_conv_10': tf.Variable(tf.zeros([48]), trainable=False),
    'bias_conv_11': tf.Variable(tf.zeros([48]), trainable=False),
    'bias_conv_12': tf.Variable(tf.zeros([48]), trainable=False),
    'bias_conv_13': tf.Variable(tf.zeros([48]), trainable=False),
    


}

In [5]:
#conv2D with bias and relu activation

class CustomConvLayer(layers.Layer):

    def __init__(self, weights, mask, biases, strides, padding='SAME'):
        
        super(CustomConvLayer, self).__init__()
        self.w = weights
        self.m = mask
        self.b = biases
        self.s = strides
        self.p = padding

        
    def call(self, inputs):
        x = tf.nn.conv2d(inputs, tf.multiply(self.w, self.m), strides=[1, self.s, self.s, 1], padding=self.p)
        x = tf.nn.bias_add(x, self.b)
        return tf.nn.relu(x)
        

#Average Pooling Layer
class CustomPoolLayer(layers.Layer):
    
    def __init__(self, k=2, padding='SAME'):#padding='VALID'):
        super(CustomPoolLayer, self).__init__()
        self.k = k
        self.p = padding
    
    def call(self, inputs):
        return tf.nn.max_pool2d(inputs, ksize=[1, self.k, self.k,1], strides=[1, self.k, self.k, 1], padding=self.p)
    
#Dense Layer with Bias
class CustomDenseLayer(layers.Layer):
    
    def __init__(self, weights, mask, bias, activation = 'relu'):
        super(CustomDenseLayer, self).__init__()
        self.w = weights
        self.b = bias
        self.a = activation
        self.m = mask
        
    def call(self, inputs):
        #print('dense w',self.w)
        #print('dense i',inputs)
        x = tf.matmul(inputs, tf.multiply(self.w, self.m))
        #print('dense x',x)
        x = tf.nn.bias_add(x, self.b)
        if self.a == 'relu':
            return tf.nn.relu(x)
        if self.a == 'softmax':
            return tf.nn.softmax(x)
        if self.a == 'sigmoid':
            return tf.nn.sigmoid(x)
    